## Bot para Trading con Simple Linear Regression

#### INSTALACION DE PYTHON

Entre a este link https://docs.continuum.io/anaconda/install/windows.html y siga las instrucciones.

#### INSTALACION DE LIBRERIAS

numpy, pandas y matplotlib: vienen instaladas con anaconda. Sklearn: http://scikit-learn.org/stable/install.html

In [1]:
#Programa Ejemplo de como programar una estrategia automatica \n",
#utilizando una regresión linear simple \n",
#Utilizando datos diarios
#Implemantados por: Juan G. Lazo Lazo , Claudio Paul Ortega Ariza \n",
#Departamento Academico de Ingenieria, Universidad del Pacífico \n",
#email: jg.lazol@up.edu.pe  \n",
#Versión: 02.0.0   \n",
# 09/11/2017  \n", 
#---------------------------------------------------------------------
import datetime as dt
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
from sklearn import linear_model
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'pandas_datareader'

In [2]:
#Importamos los datos descargados desde Yahoo Finance - Microsoft Semanales
#df = pd.read_csv('MSFT.csv', index_col=0, parse_dates=True)
df = web.get_data_yahoo('MSFT', start=dt.date(2006,11,1), end=dt.date(2016,11,25))

NameError: name 'web' is not defined

In [ ]:
df.head() #Mostramos las 5 primeras semanas

In [ ]:
df.tail() #Mostramos las 5 últimas semanas

In [ ]:
pd.isnull(df).any() #Verificamos si hay datos nulos

In [ ]:
df = df.drop(['Adj Close', 'Volume'], axis=1) #Eliminamos las columnas que no usaremos

In [ ]:
def sma(series, periods):
    '''Returns simple moving average (sma) for given series.
    Parameters are stock's yahoo finance data in DataFrame format and the number of periods of the sma'''
    sma = series.rolling(window=periods).mean()
    return sma

In [ ]:
df['sma10'] = sma(df['Close'],10)

In [ ]:
def displaced_sma(series, sma_periods, disp_periods=1):
    '''Calculates simple moving average (sma) for a stock adj close price.
    Parameters are stock's yahoo finance data in DataFrame format, the number of periods of the sma
    and the number of periods to be displaced. Default: disp_periods = 1'''
    normal_sma = series.rolling(window=sma_periods).mean()
    displaced_sma = normal_sma.shift(disp_periods)
    return displaced_sma

In [ ]:
df['dis_sma10'] = displaced_sma(df['Close'],10,1)

In [ ]:
def ema(series, periods):
    '''Calculates exponential moving average (ema) for a stock adj close price.
    Parameters are stock's yahoo finance data in DataFrame format and the number of periods of the ema'''
    series_copy = series.copy()
    series_copy[series_copy.index[periods-1]] = np.mean( series_copy[:periods] ) #first value is sum of avg losses
    series_copy = series_copy.drop(series_copy.index[:(periods-1)])
    ema = series_copy.ewm(ignore_na=False, span=periods, min_periods=0, adjust=False).mean()
    return ema

In [ ]:
df['ema9'] = ema(df['Close'],9)

In [ ]:
def macd(series, lt_period=26, st_period=12, signal_periods=9):
    '''Calculates macd line, signal and histogram for a stock adj close price.
    Parameter is stock's yahoo finance data in DataFrame'''
    #Calculates emas
    st_ema = ema(series, st_period)
    lt_ema = ema(series, lt_period)
    
    #Calculate macd line
    dif = st_ema - lt_ema
    macd_line = dif.drop(dif.index[:((26))])
    
    #Calculate macd signal
    d = macd_line
    d[d.index[(lt_period+signal_periods)-1]] = np.mean( d[:(lt_period+signal_periods)] ) #first value is sum of avg losses
    d = d.drop(d.index[:(((lt_period+signal_periods)-1))])
    macd_signal = d.ewm(ignore_na=False,span=9,min_periods=0,adjust=False).mean()
    
    #calculate macd hsit
    macd_hist = macd_line - macd_signal.fillna(0)
    
    return macd_line, macd_signal, macd_hist

In [ ]:
macd_line, macd_signal, macd_hist = macd(df['Close'])
df['macd_line'] = macd_line
df['macd_signal'] = macd_signal
df['macd_hist'] = macd_hist

In [ ]:
def bollinger_bands(series, periods=14):
    '''Calculates bollinger bands for a stock adj close price.
    Parameter is stock's yahoo finance data in DataFrame'''
    std = series.rolling(window=periods).std(ddof=0)
    sma = series.rolling(window=periods).mean()

    upper = sma + 2*std
    lower = sma - 2*std
    return upper, lower 

In [ ]:
df['bb_upper'], df['bb_lower'] = bollinger_bands(df['Close'],14)

In [ ]:
def didi_index(series):
    '''Calculates didi index for a stock adj close price.
    Parameter is stock's yahoo finance data in DataFrame'''
    sma3 = series.rolling(window=3).mean()
    sma8 = series.rolling(window=8).mean()
    sma20 = series.rolling(window=20).mean()
    
    upper = sma20 - sma8
    lower = sma3 - sma8 + sma20 - sma20 #min_periods = 20
    return upper, lower

In [ ]:
df['didi_upper'], df['didi_lower'] = didi_index(df['Close'])

In [ ]:
df = df.dropna() #Eliminamos las filas que no tengan todos los valores completos

In [ ]:
#Definimos nuestra compra: cuando el retorno es mayor a 1% en 4 días
df['Retorno>1%'] = (((df['Close'].shift(-3) - df['Close']) / df['Close']) > 0.01).astype(int)

In [ ]:
df_inputs = df.drop(['Retorno>1%'], axis=1) #Separamos las columnas que entraran al modelo como predictoras

df_label = df['Retorno>1%'] #Separamos las columnas que entraran al modelo como label

In [ ]:
sep_index = int(df.shape[0]*0.85) #Generamos el indice por el cual separaremos nuestros datos de entrenamiento y prueba

#Datos de entrenamiento
X_train = df_inputs.iloc[0:sep_index,:].values 
y_train = df_label.iloc[0:sep_index].values

#Datos de prueba
X_test = df_inputs.iloc[sep_index:,:].values
y_test = df_label.iloc[sep_index:].values

In [ ]:
model = linear_model.LinearRegression(normalize=True) #Definimos el modelo a usar para generar nuestro algoritmo de trading

In [ ]:
model.fit(X_train, y_train) #Se realiza el ajuste del modelo a los datos de entrenamiento

In [ ]:
y_predicted_train = model.predict(X_train) #Salida del modelo para los datos de entrenamiento
y_predicted_test = model.predict(X_test) #Salida del modelo para los datos de prueba

threshold = 0.5 #Valor por el cual definiremos cuando una salida es efectivamente una compra (1)

y_predicted_train[y_predicted_train > threshold] = 1
y_predicted_train[y_predicted_train <= threshold] = 0

y_predicted_test[y_predicted_test > threshold] = 1
y_predicted_test[y_predicted_test <= threshold] = 0

In [ ]:
#definiendo una función para graficar la matriz de confusión
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix");
    else:
        print('Confusion matrix, without normalization');

    print(cm);

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="orange" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
classes_names = np.array([0,1])
train_matrix = confusion_matrix(y_train,y_predicted_train)
cnf_matrix = confusion_matrix(y_test, y_predicted_test)
np.set_printoptions(precision=2)

In [ ]:
# Plot non-normalized confusion matrices
plt.figure()
plot_confusion_matrix(train_matrix, classes=classes_names, title='Confusion matrix (train data), without normalization');
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_names, title='Confusion matrix (test data), without normalization');

plt.show()

In [ ]:
#Creamos la columna en la cual pondremos las predicciones del modelo
df['Model_output'] = np.zeros(df.shape[0])
df.iloc[0:sep_index,-1] = y_predicted_train
df.iloc[sep_index:,-1] = y_predicted_test

In [ ]:
#Definimos nuestra venta: se vende 5 días luego de una compra, si no se han generado más señales de compra en ese periodo
df['Ventas'] = np.zeros(df.shape[0],)

d1 = df['Model_output'].shift(1)
d2 = df['Model_output'].shift(2)
d3 = df['Model_output'].shift(3)
#d4 = df['Model_output'].shift(4)

for index in range(df.shape[0]):
    if (df['Model_output'].iloc[index] == 0) and (d1.iloc[index] == 0) and (d2.iloc[index] == 0) and (d3.iloc[index] == 1): #and (d4.iloc[index] == 1):
        df['Ventas'].iloc[index] = 1
    else:
        df['Ventas'].iloc[index] = 0

In [ ]:
def model_eval(df):
    indexes_ventas = df.Ventas[df.Ventas == 1][::-1].index.values

    retornos = np.array([])

    for pos2_v, pos1_v in enumerate(range(indexes_ventas.shape[0]),1):
        
        precio_venta = df.Close.loc[indexes_ventas[pos1_v]]
        
        if pos2_v == indexes_ventas.shape[0]:
            precio_compra = df.Close.loc[:indexes_ventas[pos1_v]][df.Model_output == 1]        
        else:
            precio_compra = df.Close.loc[indexes_ventas[pos2_v]:indexes_ventas[pos1_v]][df.Model_output == 1]
        
        retorno = ((precio_venta - precio_compra) / precio_compra)*100
        retornos = np.append(retornos, retorno.values)
    retornos = retornos[::-1] #colocamos los retornos en orden cronológico
    
    lucro = retornos[retornos > 0] #retorno de los trades con lucro
    perdidas = retornos[retornos < 0] #retorno de los trades con lucro
    
   
    lucro_num = lucro.shape[0]
    perdida_num = perdidas.shape[0]

    #Metricas para evaluar todos los trades
    trades_num = lucro_num + perdida_num
    
    if lucro_num > 0:
        #Metricas para los trades con lucro
        lucro_perc = (lucro_num / trades_num)*100
        lucro_val = np.sum(lucro)
        lucro_promedio = lucro_val / lucro_num
        lucro_max = lucro.max()
        lucro_min = lucro.min()
    else:
        lucro_perc = "-"
        lucro_val = 0
        lucro_promedio = "-"
        lucro_max = "-"
        lucro_min = "-"
    
    if perdida_num > 0:    
        #Metricas para los trades con perdidas
        perdida_perc = (perdida_num / trades_num)*100
        perdida_val = np.sum(perdidas)
        perdida_promedio = perdida_val / perdida_num
        perdida_max = perdidas.max()
        perdida_min = perdidas.min()
    else:
        perdida_perc = "-"
        perdida_val = 0
        perdida_promedio = "-"
        perdida_max = "-"
        perdida_min = "-"       
    
    #Metricas para evaluar todos los trades
    retorno_val = lucro_val + perdida_val
    
    strategy = pd.DataFrame(data=np.array([trades_num, retorno_val]), index=['# de trades','% de retorno'], columns=['metricas'])
    strategy = strategy.round(2)
    
    lucro_perdidas =  np.array([[lucro_num, perdida_num],
                                [lucro_perc, perdida_perc],
                                [lucro_val, perdida_val],
                                [lucro_promedio, perdida_promedio],
                                [lucro_min, perdida_min],
                                [lucro_max, perdida_max]])
    
    index = ['# de trades','% de trades', '% de retorno', '% promedio por trade', 'minimo trade', 'maximo trade']
    
    columns = ['c/ lucro','c/ perdidas']
    
    metrics = pd.DataFrame(data=lucro_perdidas, index=index, columns=columns)
    metrics = metrics.round(2)
    
    return metrics, strategy

In [ ]:
metrics, strategy = model_eval(df)

In [ ]:
metrics

In [ ]:
strategy

In [ ]:
df_BT = df.iloc[:sep_index,:] #Periodo de entrenamiento o Backtesting
df_New = df.iloc[sep_index:,:] #Periodo de prueba o la simulación de nuevos datos

In [ ]:
metrics_BT, strategy_BT = model_eval(df_BT)
metrics_New, strategy_New = model_eval(df_New)

In [ ]:
metrics_BT

In [ ]:
strategy_BT

In [ ]:
metrics_New

In [ ]:
strategy_New